# Setup

In [1]:
import re
from fuzzywuzzy import fuzz
import numpy as np
import pandas as pd
from csv import QUOTE_ALL
from collections import Counter, defaultdict
from dsa.query import query
from dsa.inspection import Inspection, curate
from dsa.utils import stringify, find_columns, natural_keys, replace_all, normalize_text
from datetime import datetime
pd.set_option('max_colwidth', None)

from unidecode import unidecode 
from ftfy import fix_text
path = '/Users/ianyu/downloads'

# Action Items

## Wire Diameter 
Refer to the image examples on [this sheet](https://docs.google.com/document/d/18ktwmbQLy1aM5Z0RkxrFbMkJ3RIATsK_QzuuSGjom8A/edit).


Wire Diameter: One of the biggest issues that I’ve found is that we clearly list the wire diameter range in the title of the item, and in the attributes on our PDP, but the “wireDiameter” field will only show one end of the range, typically the smaller end. In the below example, we see that it is a 16 – 14 AWG terminal, but “wireDiameter” just says 16 AWG. In looking at the available attribute values on the enrich viewer, it looks like this happens a ton. 91 skus have a “16 AWG” value. Of those 91, only 2 actually have an individual 16 AWG size. 3 of the skus should be 12 – 10 AWG. The remaining 86 should be 16 – 14 AWG. Chances are high that the 79 in 22 AWG and the 66 in the 12 AWG have the same issue.


Additionally, there are 126 skus where “n/a” was selected. I spot checked about 15 of these, and every single one of those 15 had a wire diameter value loaded in our PDP, had the wire diameter in the title of the product, or both.
There were also 5 instances of the “Diameter” Attribute being used, and only 2 of them were accurate (.084 in). The other 3 were duplicated values from wire diameter.


In [9]:
df = query(method='curated', snake_case_name = 'wire_diameter', customer_id = 137)

In [10]:
df = pd.concat([df,pd.json_normalize(df['custom_fields'])[['WireDiameter']]], axis=1)

In [11]:
df = df[['external_id', 'product_name', 'long_desc', 'url', 'WireDiameter', 'value']]

In [12]:
na = df[~df['WireDiameter'].isna()&(df['value']=='n/a')]

In [13]:
def clean(x):
    if re.search('#', x):
        return 'n/a'
    elif re.search('to', x):
        x = x
    elif re.search('[A-Za-z]', x):
        x = re.sub('[A-Za-z\s\(\)]','', x)
        x = x.replace(',', ' to ')
    else:
        x = x
    return x + ' AWG'

In [14]:
na['Q:wire_diameter'] = na['WireDiameter'].apply(lambda x: clean(x))

<ipython-input-14-2869676c49e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  na['Q:wire_diameter'] = na['WireDiameter'].apply(lambda x: clean(x))


In [8]:
today = str(datetime.today().date())
df[df['value']=='Hi-Lo'][['external_id','value']].rename(
    columns={'value':'Q:thread_type'}).to_csv(f'/Users/ianyu/downloads/Kimball_thread_type_{today}.csv',
                                             index=False, quoting=QUOTE_ALL)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ianyu/downloads/Kimball_thread_type_2021-11-02.csv'

In [ ]:
today = str(datetime.today().date())
na[['external_id', 'Q:wire_diameter']].to_csv(f'/Users/ianyu/downloads/Kimball_wire_diameter_{today}.csv',
                                             index=False, quoting=QUOTE_ALL)

In [ ]:
query(customer_id = 64, method = 'curated', snake_case_name = 'color')

## Thread Type

In [15]:
curated = query(method='curated', customer_id = 137, snake_case_name = 'thread_type')

In [16]:
df = pd.concat([curated,pd.json_normalize(curated['custom_fields'])['ThreadType']], axis=1)

In [17]:
df = df[['external_id','product_name','long_desc','value','ThreadType']]

In [18]:
from dsa.inspection import curate

In [19]:
curate(df, 'thread_type', '(?i)hi[\-\s]lo', filter_=True)

Empty DataFrame
Columns: [external_id, product_name, long_desc, value, ThreadType, Q:thread_type]
Index: []

In [20]:
Counter(df['ThreadType'])

Counter({nan: 4809,
         'Coarse (USS)': 6026,
         'Coarse (USS)&Fine (SAE)': 69,
         'NPT': 86,
         'Metric': 218,
         'Fine (SAE)': 1890,
         'Fine': 36,
         'Coarse': 86,
         'Fine Pitch (Fine Thread)': 203,
         'Hi-Lo': 54,
         'Regular Pitch (Coarse Thread)': 2304,
         'Very Fine Pitch (Very Fine Thread)': 42,
         'Metric Taper Thread': 19,
         'Self-Tapping': 8,
         'Metric&British Pipe': 3,
         'Coarse (USS), Fine (SAE)&NPT': 4,
         'Regular&Fine Pitches': 3,
         'External': 13,
         'Internal': 16,
         'Drive': 4,
         'UNF': 3,
         '1/4"-20': 1})

In [21]:
list_ = []
for value, custom in zip(df['value'], df['ThreadType']):
    if custom =='Hi-Lo':
        list_.append('Hi-Lo')
    else:
        list_.append(value)

In [22]:
df['value']=list_

In [23]:
today = str(datetime.today().date())
df[df['value']=='Hi-Lo'][['external_id','value']].rename(
    columns={'value':'Q:thread_type'}).to_csv(f'/Users/ianyu/downloads/Kimball_thread_type_{today}.csv',
                                             index=False, quoting=QUOTE_ALL)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ianyu/downloads/Kimball_thread_type_2021-11-02.csv'

## Color-Coded

In [2]:
curated = query(method='curated', customer_id = 137, snake_case_name = 'color')

In [3]:
cp = query(
    '''SELECT * FROM categories_products WHERE customer_id = 137''')
pc = cp[['product_id', 'category_id', 'created_at', 'updated_at']]

In [ ]:
pc

In [ ]:
def in_cat(id_, target):
    try:
        return any(cat in pc[pc['product_id'] == id_]['category_id'].to_list() for cat in target)
    except:
        return False

In [ ]:
curated['category_id'] = curated['product_id'].apply(lambda x: in_cat(x, [24311019]))

In [ ]:
wipe = curated[curated['category_id']==True][['external_id','value']]

In [ ]:
wipe['value'] = ''

In [ ]:
wipe.rename(columns={'value':'Q:color'}, inplace=True)

In [ ]:
today = str(datetime.today().date())
wipe.to_csv(f'/Users/ianyu/downloads/Kimball_color_{today}.csv',
                                             index=False, quoting=QUOTE_ALL)

## Dialectic Strength

In [ ]:
curated = query(method = 'curated', customer_id = 137, snake_case_name = 'dielectric_strength')

In [ ]:
curated = pd.concat([curated, pd.json_normalize(curated['custom_fields'])['DielectricStrength']], axis=1)
curated = curated[['external_id','product_name','long_desc','value','DielectricStrength']]

In [ ]:
na = curated[(curated['value']!='n/a')&(curated['DielectricStrength'].isna())]

In [ ]:
na['value'] = 'n/a'

In [ ]:
today = str(datetime.today().date())
na[['external_id', 'value']].rename(columns={'value':'Q:dielectric_strength'}).to_csv(
    f'/Users/ianyu/downloads/Kimball_dielectric_strength_{today}.csv',index=False, quoting=QUOTE_ALL)

## Finish

In [ ]:
curated = query(method = 'curated', customer_id = 137, snake_case_name = 'finish')

In [ ]:
curated = pd.concat([curated, pd.json_normalize(curated['custom_fields'])['Finish']], axis=1)
curated = curated[['external_id','product_name','long_desc','value','Finish']]

In [ ]:
na = curated[(curated['value']=='n/a')&~(curated['Finish'].isna())]

In [ ]:
na['Finish'] = na['Finish'].apply(lambda x: replace_all(x, {'ALTiN Coated': ['Black Titanium Aluminum Nitride', 'TiALN'],
'Black': ['Black/Silver'],
'Nitride': ['NOMO Surface Treatment'],
'''["Zinc Plated","Yellow Zinc"]''': ['Zinc Yellow Plated'],
'''["Zinc Plated","Yellow Dichromate"]''': ['Zinc, Yellow Dichromate'],
'Color': ['Red Color Coated'], 'n/a': ['Corrosion Resistant', 'Flat']}))

In [ ]:
today = str(datetime.today().date())
na[['external_id', 'Finish']].rename(columns={'value':'Q:finish'}).to_csv(
    f'/Users/ianyu/downloads/Kimball_finish_{today}.csv',index=False, quoting=QUOTE_ALL)

## Dimensions

In [ ]:
df = pd.read_csv('data/electrical_dim.csv')
df.drop(['active','created_at', 'category_ids', 'category_names', 'groupings'], axis=1, inplace=True)

In [ ]:
curated = query(method='curated', customer_id = 137, snake_case_name = 'length')

In [ ]:
df.columns

In [ ]:
df[(df['C:Length'].isna())&(df['C:Width'].isna())]

## Measurement Type

In [ ]:
c = 'ThreadType'
q = 'Q:measurement_type'
curated = query(method='curated', customer_id = 137, snake_case_name = 'measurement_type')
curated = pd.concat([curated, pd.json_normalize(curated['custom_fields'])[c]], axis=1)
curated = curated[['external_id', 'product_name', 'long_desc', c, 'value']]
curated[c] = curated[c].fillna('')

In [ ]:
list_ = []
for product, desc, custom in zip(curated['product_name'], curated['long_desc'], curated[c]):
    text = normalize_text(product + desc + custom)
    list_.append(re.findall('''(\d[Mm]{2}|\d[Cc][Mm]|USS|NPT|SAE|[Mm]etric|\b[Cc]oarse\b|\b[Ff]ine\b)''', text))
curated['Q:measurement_type'] = list_

In [ ]:
def clean(x):
    x = [re.sub('''([Ff]ine)''', 'SAE', y) for y in x]
    x = [re.sub('''([m]etric|\dmm)''', 'Metric', y) for y in x]
    x = [re.sub('''([Cc]oarse)''', 'USS', y) for y in x]
    x = list(set(x))
    if len(x) == 0:
        x = 'n/a'
    elif len(x) == 1:
        x = x[0]
    return stringify(x)

In [ ]:
curated[q] = curated[q].apply(lambda x: clean(x))

In [ ]:
curated[curated[q].str.contains('SAE')].sample(50)

In [ ]:
sum(curated.duplicated('external_id'))

In [ ]:
curated[curated[q].str.contains('NPT')]

In [ ]:
today = str(datetime.today().date())
curated[['external_id', q]].to_csv(
    f'/Users/ianyu/downloads/Kimball_{q.replace("Q:","")}_{today}.csv',index=False, quoting=QUOTE_ALL)

## Grit

In [52]:
df = query(method = 'curated', snake_case_name = 'grit', customer_id = 137)

In [53]:
custom = pd.json_normalize(df['custom_fields'])

In [54]:
df = pd.concat([df, custom['Grit']], axis=1)

In [55]:
df['Grit'].fillna('n/a', inplace=True)

In [57]:
df['Grit'].unique()

array(['n/a', '60, 80, 120', '80', 'Medium&Coarse', '40, 60, 80',
       '80, 120, 180, 240', '500', '36, 50, 80', 'Assortment', '120',
       'Fine', '60', '180', '320', '240', '400', '150',
       '80 J, 120 J, 150 J', '180 J, 240 J, 320 J', '80, 120, 180', '36',
       'Coarse', 'Medium', '40', 'Very Fine', '24', '220', '50', '100',
       'Extra Coarse', '54', '600', 'Very Coarse', 'Silicon Carbide',
       'Super Fine', 'Assorted', '46', '10', '280', '360'], dtype=object)

In [71]:
def clean(x):
    if (x == 'n/a') or (x == 'Silicon Carbide'):
        return 'n/a'
    x = replace_all(x, {'':[' J'], ',':['&',', ']})
    return x

In [72]:
fixes = df[(df['value']=='n/a')&(df['Grit']!='n/a')]

In [77]:
fixes['Q:grit'] = fixes['Grit'].apply(lambda x: clean(x))

/Users/ianyu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [80]:
today = str(datetime.today().date())
today = today.replace('-', '_')
fixes[['external_id','Q:grit']].to_csv(f'{path}/kimball_grit_{today}.csv', quoting=QUOTE_ALL, index=False)

# Curation

## Dielectric Strength


In [38]:
a = Inspection(customer_id = 137, snake_case_name = 'dielectric_strength', method='curated')
c = 'DielectricStrength'
q = 'Q:dielectric_strength'

In [40]:
x=a.cf_
x

In [33]:
a.expand(['DielectricStrength'], inplace=True)
a.lean(['buckets','DielectricStrength'], inplace=True)

TypeError: 'NoneType' object is not subscriptable

In [41]:
a.df


buckets  \
0                                                               Tapes   
1                                                               Tapes   
2                                                               Tapes   
3                                                               Tapes   
4                                                               Tapes   
...                                                               ...   
1834  Cables, Cords & Cord Management Replacement Parts & Accessories   
1835  Cables, Cords & Cord Management Replacement Parts & Accessories   
1836  Cables, Cords & Cord Management Replacement Parts & Accessories   
1837  Cables, Cords & Cord Management Replacement Parts & Accessories   
1838                                                       Lubricants   

      bucket_id  product_id family_id external_id  \
0          8833  1173635449      None       26003   
1          8833  1173635845      None       83566   
2          8833  1173635985      None       26004   
3          8833  1173636319      None       26007   
4          8833  1173636368      None       26006   
...         ...         ...       ...         ...   
1834       9612  1357762682      None     PD95156   
1835       9612  1357762683      None     PD95212   
1836       9612  1357762684      None     PDSN654   
1837       9612  1357762685      None     PMSN520   
1838       7270  1365849434      None       80990   

                                                        product_name  \
0                             ScotchCode™ Wire Marker Tape Refill #3   
1                                                    3M Masking Tape   
2                             ScotchCode™ Wire Marker Tape Refill #4   
3                             ScotchCode™ Wire Marker Tape Refill #7   
4                             ScotchCode™ Wire Marker Tape Refill #6   
...                                                              ...   
1834       Pro-Tech Commercial Grade Heat Shrink Ring Terminal 8 AWG   
1835       Pro-Tech Commercial Grade Heat Shrink Ring Terminal 8 AWG   
1836      Pro-Tech Commercial Grade Heat Shrink Butt Connector 8 AWG   
1837  Pro-Tech Commercial Grade Heat Shrink Butt Connector 24-22 AWG   
1838                                                     Fluid Film®   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Provides a fast, convenient method for marking wire or cable. Epoxy film tape will not smear. Acrylic adhesive has excellent solvent/oil resistance.   
1                                                                                                                                                                                                                                                            

In [37]:
# custom_field_df=pd.json_normalize(a.df['custom_fields'])
# # custom_field_df = pd.json_normalize(df['custom_fields'])
# fields = ['Dia']
# length = pd.concat([a.df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# print(len(df))

In [35]:
x=a.cf_
x

In [ ]:
[x for x in list(a.df[c].unique()) if x not in '''10,000 Volts
1,000 V/mil
10,625 Volts
1,100 V/mil
1,250 Volts
17,300 Volts
26,200 Volts
33,000 Volts
39,000 Volts
39,000+ Volts
400 V/mil
40,100 Volts
40,700 Volts
430 Volts
4,740 Volts
5,000 Volts
520 V/mil
69,000 Volts
700 V/mil
8,750 Volts'''.splitlines()]

In [ ]:
test = [(bucket, value) for bucket, value in zip(a.df[~a.df[c].isna()]['buckets'], a.df[~a.df[c].isna()][c])]

In [ ]:
test = list(set(test))

In [ ]:
d1 = defaultdict(list)

for k, v in l:
    d1[k].append(v)

d = dict((k, tuple(v)) for k, v in d1.items())

In [ ]:
dict_ = defaultdict(list)
for item in test:
    dict_[item[0]].append(item[1])
dict_ = {k: v for k, v in dict_.items()}

In [ ]:
for k, v in dict_.items():
    print(f'''{k}: {', '.join(v)}''')

In [ ]:
Counter(a.df[c])

## Bond Time

In [42]:
a = Inspection(customer_id = 137, snake_case_name = 'bond_time')
c = 'Fasten_BondTime'
q = 'Q:bond_time'
a.expand([c], inplace=True)
a.lean([c], inplace=True)

TypeError: 'NoneType' object is not subscriptable

In [43]:
curated = query(method = 'curated', customer_id = 137, snake_case_name = 'bond_time')
curated = pd.concat([curated, pd.json_normalize(curated['custom_fields'])], axis=1)
curated = curated[['external_id', 'product_name', 'long_desc', 'value', c]]

In [ ]:
a.

In [ ]:
a.df[a.df[c]=='Various']

In [ ]:
curated[curated[c]=='10 sec. - 30 min.']

In [ ]:
a.df[a.df[c].isna()]

In [ ]:
{(custom, value) for custom, value in zip(curated[c], curated['value'])}

In [ ]:
def fixes(*args):
    list_ = []
    for arg in args:
        list_+=list(arg['external_id'])
    return list_

In [ ]:
a.df[a.df[c]=='Instant']

In [ ]:
instant = fixes(curated[(curated[c].isna())&(curated['long_desc'].str.contains('(?i)instant'))]
     ,curated[curated[c]=='Instant'])
instant = pd.DataFrame({'external_id': instant})
instant[q] = 'Instant'

In [ ]:
Counter(a.df[c])

In [ ]:
uncurated = a.df[~(a.df[c].isna())&~(a.df[c]=='Seconds')]

In [ ]:
uncurated[c] = uncurated[c].apply(lambda x: replace_all(x, {'': '.', ' to ': '-'}).replace('hrs', 'hr'))

In [ ]:
uncurated[q] = uncurated[c]

In [ ]:
uncurated1 = uncurated[['external_id', q]]

In [ ]:
instant.shape

In [ ]:
final = pd.concat([uncurated1, instant])

In [ ]:
today = str(datetime.today().date())
final[['external_id', q]].to_csv(
    f'/Users/ianyu/downloads/Kimball_{q.replace("Q:","")}_{today}.csv',index=False, quoting=QUOTE_ALL)

## Cure Time

In [ ]:
c = 'CureTime'
curated = query(method = 'uncurated', customer_id = 137, snake_case_name = 'cure_time')
curated = pd.concat([curated, pd.json_normalize(curated['custom_fields'])], axis=1)
curated = curated[['external_id', 'product_name', 'long_desc', 'value', c]]

In [ ]:
Counter(curated[c])

In [ ]:
curated[curated[c].isna()]

## Brand

In [8]:
c = 'Brand'
q = 'brand'
a = Inspection(customer_id = 137, snake_case_name = 'brand')

In [9]:
brands = query(f'''
SELECT id, brand FROM products
WHERE customer_id = 137
AND id in ({', '.join(list(a.df['product_id'].apply(lambda x: str(x))))})
''')

In [10]:
brands = brands.rename(columns={'id':'product_id'})

In [11]:
a.df = pd.merge(a.df, brands)

In [12]:
a.lean(['brand'], inplace=True)

In [13]:
a.df['brand'].unique()

array([None, 'Kimball Midwest', 'Kimball Midwest Approved Vendor',
       'SmartWasher'], dtype=object)

In [14]:
output = a.df[~a.df['brand'].isna()].rename(columns={'brand':'Q:brand'})[['external_id', 'Q:brand']]

In [40]:
today = str(datetime.today().date())
output.to_csv(f'{path}')

datetime.datetime

## Flammable

In [16]:
customer_id = 137
snake_case_name = 'flammable_non_flammable'
a = Inspection(customer_id = customer_id, snake_case_name = snake_case_name)

In [17]:
a.total

89

# Electrical QA

In [ ]:
def compare(array1, array2, threshold, method='ratio'):
    methods = {'ratio': fuzz.ratio, 'partial': fuzz.partial_ratio}
    list_ = []
    for a1, a2 in zip(array1, array2):
        if methods[method](str(a1), str(a2)) < threshold:
            list_.append((a1, a2))
    return list_

In [ ]:
def truncate(c, q):
    return df[(df[c]!='n/a')&(df[q]=='n/a')][['external_id', 'name', 'description', 'url', c, q]]

def count_truncate(c, q, t):
    return Counter(truncate(c, q)[t])

def export(df, q):
    today = str(datetime.today().date())
    q_type = re.search("Q\:(\w{1,})", q).group(1)
    df[['external_id', q]].to_csv(f'/Users/ianyu/downloads/Kimball_{q_type}_{today.replace("-","_")}.csv', quoting=QUOTE_ALL, index=False)

In [ ]:
df = pd.read_csv('data/electrical_fix.csv')

In [ ]:
df.fillna('n/a', inplace=True)

In [ ]:
df.shape[1]

In [ ]:
df.columns[130:]

In [ ]:
c = 'C:TensileStrength_P_S_I__'
q = 'Q:tensile_strength'

In [ ]:
compare(df[c], df[q], 80, method = 'partial')

In [ ]:
def clean(x):
    if re.search('250V', x):
        x = '250'
    elif re.search('36', x):
        x = '36'
    elif re.search('28', x):
        x = '28'
    elif re.search('12', x):
        x = '12'
    return x + 'V'

In [ ]:
count_truncate(c, q, c)

In [ ]:
truncate(c, q)

In [ ]:
na[q] = na[c].apply(lambda x: clean(x))

In [ ]:
export(na, q)

# API

In [4]:
import requests

In [5]:
jwt = '''eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpZCI6Nzc1LCJuYW1lIjoiSWFuIFl1IiwiZW1haWwiOiJpYW4ueXVAZ3JvdXBieWluYy5jb20iLCJwZXJtaXNzaW9ucyI6WyJBY2Nlc3MgRGFzaGJvYXJkIiwiTWFuYWdlIFVzZXJzIiwiTWFuYWdlIFJvbGVzIiwiVmlldyBKb2JzIiwiVmlldyBMb2dnaW5nIiwiVmlldyBTZXJ2aWNlcyIsIkRldmVsb3BlciIsIlZpZXcgQ3VzdG9tZXJzIiwiQWRkIEN1c3RvbWVycyIsIk1vZGlmeSBDdXN0b21lcnMiLCJJbXBvcnQgQ3VzdG9tZXJzIiwiTWFuYWdlIFNGVFAgQ3JlZGVudGlhbHMiLCJWaWV3IEN1c3RvbWVyIERhdGEiLCJWaWV3IFByb2R1Y3QgRGV0YWlscyIsIlByb2Nlc3MgRmVlZHMiLCJWaWV3IFNjb3V0IEZpbGUiLCJNb2RpZnkgUHJvZHVjdCBEYXRhIiwiUHVibGlzaCBQcm9kdWN0IERhdGEiLCJSZWNlaXZlIFVuY3VyYXRlZCBQcm9kdWN0IEVtYWlscyIsIlJlY2VpdmUgU3RhbGUgSm9iIEVtYWlscyIsIkRlbGV0ZSBDdXJhdGVkIERhdGEiLCJTaG91bGQgTm90IEdldCBEYXNoYm9hcmQiLCJNaWdyYXRlIEN1cmF0aW9uIiwiV2hvbGUgRW5jaGlsYWRhIiwiU3VibWl0IFByb2R1Y3QgRmVlZGJhY2siLCJWaWV3IFJlcG9ydHMiLCJSdW4gUmVwb3J0IiwiR2VuZXJhdGUgVmlld3MiLCJHZW5lcmF0ZSBDYXRlZ29yeSBXaGl0ZWxpc3QiLCJWaWV3IEdsb2JhbCBUYXhvbm9teSIsIkVkaXQgR2xvYmFsIFRheG9ub215IiwiVmlldyBHbG9iYWwgQ2xhc3NpZmljYXRpb24gUnVsZXMiLCJFZGl0IEdsb2JhbCBDbGFzc2lmaWNhdGlvbiBSdWxlcyIsIlZpZXcgQ3VzdG9tZXIgU3RyYXRlZ3kiLCJFZGl0IEN1c3RvbWVyIFN0cmF0ZWd5IiwiRWRpdCBDdXN0b21lciBTdHJhdGVneSBpbiBQcmV2aWV3IiwiQXBwcm92ZSBDdXN0b21lciBTdHJhdGVneSIsIkVkaXQgR3VpZGVsaW5lcyBJbiBDdXJhdGlvbiIsIlZpZXcgR3VpZGVsaW5lcyIsIkVkaXQgR3VpZGVsaW5lcyIsIlZpZXcgQ3VyYXRpb24gSm9icyIsIk1hbmFnZSBDdXJhdGlvbiBKb2JzIiwiRWRpdCBDdXJhdGlvbiBUYXNrIEZpZWxkcyIsIlBlcmZvcm0gQ3VyYXRpb24gVGFza3MiLCJQZXJmb3JtIFFBIEN1cmF0aW9uIFRhc2tzIiwiUHJldmlldyBDdXJhdGlvbiBUYXNrcyIsIlJlY2VpdmUgQ3VyYXRpb24gVGFzayBFbWFpbHMiLCJSZWNlaXZlIFN0cmF0ZWd5IEFwcHJvdmFsIEVtYWlscyIsIkNoYW5nZSBQcm9kdWN0IEJ1Y2tldCIsIk1hbmFnZSBDdXJhdGlvbiBSdWxlcyIsIlZpZXcgQ3VyYXRpb24gUGVyZm9ybWFuY2UiLCJNYW5hZ2UgTWFjaGluZSBDdXJhdGlvbiIsIlZpZXcgSW5zaWdodHMiLCJWaWV3IFVzYWdlIERhc2hib2FyZCIsIlZpZXcgQXR0cmlidXRlIEludGVsbGlnZW5jZSJdLCJjdXN0b21lcklkcyI6Wzg3LDExNiw3MSw2OCw1MSw3MCw1Miw4MCwxMzIsODQsOTIsMTAxLDY5LDExNCwxMTUsNjAsOTcsMTEyLDEwOCwyMiw1OSwxMzUsNjUsMTI3LDEyNCw5OCwxMDMsMTEsNDQsNDIsMTE3LDEyMSw4OCw4Miw0MCwxMTMsMTE5LDEyNSw0Myw5LDEyMCwxNSw3OSwyNiw0OCw4NSw3Miw5NSw1NywxOSw2MSw4MSw3NywzMCwyMSwxMzEsMTcsMzcsMTA0LDUsNTYsOTEsNzQsNTQsMjksNCwxMzgsMzQsOTYsNjcsODMsNjMsMTAsOTAsMTA1LDM1LDQ1LDYsMTA3LDEzNCw4NiwzOSw4OSw5MywzNiwzMSw1MCwxMDIsMTQsNjYsMTA5LDEzLDExOCwxMzMsMTExLDIsMTYsNjIsNzUsMTIzLDEyOCwxMjYsOTksMTQyLDQxLDQ2LDUzLDMyLDcsMTAwLDM4LDEzNiwxMzksMTQwLDEyLDEzNyw3OCwyNCwyNSw5NCwxMjIsMTQxLDQ5LDQ3LDIwLDMzLDEsNzYsMTA2LDE4LDY0LDExMCw1NSwxMjksMTMwLDI3LDIzLDU4LDhdLCJpYXQiOjE2MzM0NTU1MDcsImV4cCI6MTYzNDA2MDMwN30.sEHwHGdic0WX4lzUmX7OGY8KJa36J6veYj5n1RWNP7E'''
dev_jwt = '''eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpZCI6Nzc1LCJuYW1lIjoiSWFuIFl1IiwiZW1haWwiOiJpYW4ueXVAZ3JvdXBieWluYy5jb20iLCJwZXJtaXNzaW9ucyI6WyJBY2Nlc3MgRGFzaGJvYXJkIiwiTWFuYWdlIFVzZXJzIiwiTWFuYWdlIFJvbGVzIiwiVmlldyBKb2JzIiwiVmlldyBMb2dnaW5nIiwiVmlldyBTZXJ2aWNlcyIsIkRldmVsb3BlciIsIlZpZXcgQ3VzdG9tZXJzIiwiQWRkIEN1c3RvbWVycyIsIk1vZGlmeSBDdXN0b21lcnMiLCJJbXBvcnQgQ3VzdG9tZXJzIiwiTWFuYWdlIFNGVFAgQ3JlZGVudGlhbHMiLCJWaWV3IEN1c3RvbWVyIERhdGEiLCJWaWV3IFByb2R1Y3QgRGV0YWlscyIsIlByb2Nlc3MgRmVlZHMiLCJWaWV3IFNjb3V0IEZpbGUiLCJNb2RpZnkgUHJvZHVjdCBEYXRhIiwiUHVibGlzaCBQcm9kdWN0IERhdGEiLCJSZWNlaXZlIFVuY3VyYXRlZCBQcm9kdWN0IEVtYWlscyIsIlJlY2VpdmUgU3RhbGUgSm9iIEVtYWlscyIsIkRlbGV0ZSBDdXJhdGVkIERhdGEiLCJTaG91bGQgTm90IEdldCBEYXNoYm9hcmQiLCJNaWdyYXRlIEN1cmF0aW9uIiwiV2hvbGUgRW5jaGlsYWRhIiwiU3VibWl0IFByb2R1Y3QgRmVlZGJhY2siLCJWaWV3IFJlcG9ydHMiLCJSdW4gUmVwb3J0IiwiR2VuZXJhdGUgVmlld3MiLCJHZW5lcmF0ZSBDYXRlZ29yeSBXaGl0ZWxpc3QiLCJWaWV3IEdsb2JhbCBUYXhvbm9teSIsIkVkaXQgR2xvYmFsIFRheG9ub215IiwiVmlldyBHbG9iYWwgQ2xhc3NpZmljYXRpb24gUnVsZXMiLCJFZGl0IEdsb2JhbCBDbGFzc2lmaWNhdGlvbiBSdWxlcyIsIlZpZXcgQ3VzdG9tZXIgU3RyYXRlZ3kiLCJFZGl0IEN1c3RvbWVyIFN0cmF0ZWd5IiwiRWRpdCBDdXN0b21lciBTdHJhdGVneSBpbiBQcmV2aWV3IiwiQXBwcm92ZSBDdXN0b21lciBTdHJhdGVneSIsIkVkaXQgR3VpZGVsaW5lcyBJbiBDdXJhdGlvbiIsIlZpZXcgR3VpZGVsaW5lcyIsIkVkaXQgR3VpZGVsaW5lcyIsIlZpZXcgQ3VyYXRpb24gSm9icyIsIk1hbmFnZSBDdXJhdGlvbiBKb2JzIiwiRWRpdCBDdXJhdGlvbiBUYXNrIEZpZWxkcyIsIlBlcmZvcm0gQ3VyYXRpb24gVGFza3MiLCJQZXJmb3JtIFFBIEN1cmF0aW9uIFRhc2tzIiwiUHJldmlldyBDdXJhdGlvbiBUYXNrcyIsIlJlY2VpdmUgQ3VyYXRpb24gVGFzayBFbWFpbHMiLCJSZWNlaXZlIFN0cmF0ZWd5IEFwcHJvdmFsIEVtYWlscyIsIkNoYW5nZSBQcm9kdWN0IEJ1Y2tldCIsIk1hbmFnZSBDdXJhdGlvbiBSdWxlcyIsIlZpZXcgQ3VyYXRpb24gUGVyZm9ybWFuY2UiLCJNYW5hZ2UgTWFjaGluZSBDdXJhdGlvbiIsIlZpZXcgSW5zaWdodHMiLCJWaWV3IFVzYWdlIERhc2hib2FyZCIsIlZpZXcgQXR0cmlidXRlIEludGVsbGlnZW5jZSJdLCJjdXN0b21lcklkcyI6Wzg3LDExNiw3MSw2OCw1MSw3MCw1Miw4MCwxMzIsODQsOTIsMTAxLDY5LDExNCwxMTUsNjAsOTcsMTEyLDEwOCwyMiw1OSwxMzUsNjUsMTI3LDEyNCw5OCwxMDMsMTEsNDQsNDIsMTE3LDEyMSw4OCw4Miw0MCwxMTMsMTE5LDEyNSw0Myw5LDEyMCwxNSw3OSwyNiw0OCw4NSw3Miw5NSw1NywxOSw2MSw4MSw3NywzMCwyMSwxMzEsMTcsMzcsMTA0LDUsNTYsOTEsNzQsNTQsMjksNCwxMzgsMzQsOTYsNjcsODMsNjMsMTAsOTAsMTA1LDM1LDQ1LDYsMTA3LDEzNCw4NiwzOSw4OSw5MywzNiwzMSw1MCwxMDIsMTQsNjYsMTA5LDEzLDExOCwxMzMsMTExLDIsMTYsNjIsNzUsMTIzLDEyOCwxMjYsOTksMTQyLDQxLDQ2LDUzLDMyLDcsMTAwLDM4LDEzNiwxMzksMTQwLDEyLDEzNyw3OCwyNCwyNSw5NCwxMjIsMTQxLDQ5LDQ3LDIwLDMzLDEsNzYsMTA2LDE4LDY0LDExMCw1NSwxMjksMTMwLDI3LDIzLDU4LDhdLCJpYXQiOjE2MzM0NTU1MDcsImV4cCI6MTYzNDA2MDMwN30.sEHwHGdic0WX4lzUmX7OGY8KJa36J6veYj5n1RWNP7E'''

In [6]:
r = requests.get('https://enrich-legacy-api.dev.groupbycloud.com/api/v3/strategyBuckets', 
             headers = {"Authorization": f'Bearer {dev_jwt}'}, params={'filter': '{"strategyVersionId"}: 20'})
r.status_code

401

In [7]:
r = requests.get('https://enrich-legacy-api.groupbycloud.com/api/v3/strategyBuckets', 
             headers = {"Authorization": f'Bearer {jwt}'}, params={'filter': ''})
r.status_code

401